Assumption: The earnings date is taken as the ex-date

1. Get list of company tickers from finviz
2. Get earnings data report for each company in `companies`
3. Make a DF row for each company row

Columns to populate:
For each company: For each earnings
1. compute gap%
2. find C1
3. find O
4. find market cap
5. find distance from 52W high
6. find recency of 52W high
7. trading volume near announcement (at exdate)
8. trading volume before announcement (at c1)

FORMULAE:
1. Market cap = Total shares outstanding * price at that date
2. 52W_HIGH = 
3. 52W_RECENCY = 
4. 


CHANGES:
1. Change 52week defintion to 252 days instead of 364 days.
2. for the first week, do min (52 weeks, available weeks)


In [3]:
import pandas as pd
stocks = pd.read_csv('stocks/us_stocks.csv')

In [4]:
stocks

,Unnamed: 0,Ticker,Company,Sector,Industry,Market Cap,Perf Month,Perf Quart,Perf Half,Volatility M,SMA20,SMA50,Avg Volume,Price,Earnings
0,0,RKDA,"Arcadia Biosciences, Inc.",Consumer Defensive,Packaged Foods,2.277300e+08,26.3885,23.6418,9.0000,0.1294,29.5821,27.7193,282180.0,8.60,-
1,1,CDIO,"Cardio Diagnostics Holdings, Inc.",Healthcare,Biotechnology,6.422000e+07,5.5347,2.3000,-0.3353,0.3303,3.4437,4.2354,2130000.0,6.60,-
2,2,MSGM,Motorsport Games Inc.,Communication Services,Electronic Gaming & Multimedia,8.527000e+07,2.3460,0.6826,0.4679,0.3909,-0.3766,0.1209,1400000.0,8.80,-
3,3,SLQT,"SelectQuote, Inc.",Financial,Insurance Brokers,4.333300e+08,2.0459,2.6313,0.7786,0.1783,0.3823,1.1536,2590000.0,2.49,Feb 07/b
4,4,AMAM,Ambrx Biopharma Inc.,Healthcare,Biotechnology,2.953200e+08,1.9303,11.2708,2.2363,0.2325,0.9885,1.4564,6470000.0,5.89,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5230,5230,MARXU,Mars Acquisition Corp.,Financial,Shell Companies,6.469000e+07,NaN,NaN,NaN,NaN,0.0032,0.0032,565230.0,10.21,-
5231,5231,HSAI,Hesai Group,Consumer Cyclical,Auto Parts,2.290000e+09,NaN,NaN,NaN,NaN,-0.1266,-0.1266,1160000.0,18.25,-
5232,5232,GPCR,Structure Therapeutics Inc.,Healthcare,Biotechnology,8.392400e+08,NaN,NaN,NaN,NaN,-0.0284,-0.0284,180080.0,22.93,-
5233,5233,DISTU,Distoken Acquisition Corporation Unit,Financial,Shell Companies,6.626000e+07,NaN,NaN,NaN,NaN,0.0009,0.0009,480710.0,10.21,-


In [5]:
ignore_filters = [ 'Closed-End Fund - Foreign', 'Closed-End Fund - Equity', 'Closed-End Fund - Debt', 'Shell Companies', 'Exchange Traded Fund']
filtered_stocks = stocks.loc[~stocks['Industry'].isin(ignore_filters)]
filtered_stocks.to_csv('stocks/FINAL_US_STOCKS.csv', mode='w', sep=',')

In [6]:
filtered_us_stocks = pd.read_csv('stocks/FINAL_US_STOCKS.csv')

In [7]:
filtered_us_stocks.shape

(4700, 16)

In [61]:
import os
def parse_symbols(dirname):
   files = os.listdir(dirname)
   return [e[:e.index('_')] for e in files]

done = parse_symbols('earnings')

df = filtered_us_stocks.loc[~filtered_us_stocks['Ticker'].isin(done)]
del df['Unnamed: 0']
del df['Unnamed: 0.1']

In [62]:
df.to_csv('stocks/FINAL_US_STOCKS.csv')

In [63]:
df = pd.read_csv('stocks/FINAL_US_STOCKS.csv')

In [ ]:
df

In [ ]:
# GET US STOCK LIST FROM FINVIZ

from finvizfinance.screener.custom import Custom

def get_us_stocks():
    """
    Unused
    """
    fcustom = Custom()
    cols = [0, 1]
    ignore_filters = [ 'Closed-End Fund - Foreign', 'Closed-End Fund - Equity', 'Closed-End Fund - Debt', 'Shell Companies', 'Exchange Traded Fund']
    my_industries = list(set(get_fv_filter_options('Industry')) - set(ignore_filters))
#     print(my_industries)
    master_df = pd.DataFrame()
    for industry in my_industries:
        fcustom = Custom()
        cols = [0,1]
        my_filters = { 'Country': 'USA', 'Industry': industry}
        fcustom.set_filter(filters_dict=my_filters)
        df1 = fcustom.screener_view(columns=cols, sleep_sec=1, order='Ticker')
        pd.concat([master_df, df1], ignore_index=True)
        print('[+] Done {}'.format(industry))
    return master_df
#     print("stringified", ','.join(my_industries))
#     filters = { 'Country': 'USA', 'Industry': ','.join(my_industries)}
#     fcustom.set_filter(filters_dict=filters)
#     df1 = fcustom.screener_view(columns=cols, sleep_sec=1, order='Ticker')
#     return df1

def get_filters():
    fcustom = Custom()
    return fcustom.get_filters()

def write_to_file(df, filename):
    df.to_csv(rf"{filename}", header=None, sep=' ', mode='w')

def get_fv_filter_options(filtername):
    fcustom = Custom()
    return fcustom.get_filter_options(filtername)

write_to_file(get_us_stocks(), "stocks/UPDATED_US_STOCKS.txt")
print("[+] Completed Task")

In [8]:
# PACAKGE VERSION REQUIREMENTS - YAHOOQUERY
import yahooquery as yq
import requests
import urllib3

print(yq.__version__) # >=2.3.0
print(requests.__version__) # >= 2.28.2
print(urllib3.__version__) # >= 1.26.14

2.3.0
2.28.2
1.26.14


In [9]:
meta = yq.Ticker('META', asynchronous=True)

In [10]:
df = meta.history(period="max", interval="1d")

In [12]:
df.to_csv(fr"tmp/META_ADJUSTED_OHLC.csv", mode='w', sep=',', header=df.columns)

In [69]:
# GET OHLC DATA READING FROM US_STOCKS in stocks/
# Note: When this was first run, we got all data (even ETFs, index funds, etc.)

import time
import os
import sys

def parse_symbols(dirname):
   files = os.listdir(dirname)
   return [e[:e.index('_')] for e in files]

def read_symbols(filename):
    with open(filename, "r") as file:
      data = file.read().split('\n')
      data = [d.split(',')[1] for d in data]
      data.pop(0) # ignore header
    return data

def get_bulk_ohlc_data():
  symbols = list(set(read_symbols("stocks/US_STOCKS.txt")) - set(parse_symbols('ohlc/')))
  with open("logs/ohlc_errors.txt", "w") as f:
    for symbol in symbols:
        try:
          company = yq.Ticker(symbol, asynchorous=True)
          df = company.history(period="max", interval="1d")
          df.to_csv(fr"ohlc/{symbol}_ADJUSTED_OHLC.csv", mode='w', sep=',', header=df.columns)
          print("[+] Done " + symbol)
          time.sleep(2) # 2000 requests/hour
        except Exception as e:
          print("An exception occured: {}".format(e))
          f.write(str(e) + "\n")
          pass

# get_bulk_ohlc_data()
        

In [6]:
import time
import requests
import pandas as pd
import math

# MAKE DIRECTORIES earnings/, logs/, income_stmts/ before running get_data()
# This reads from FINAL_US_STOCKS.csv in stocks/
# Make sure it exists at the desired path

BASE_URL = 'https://alphavantage.co'
AV_API_KEY = 'IHLNUQ0G66C0MJID'
# ignore
AV_API_KEY_2 = 'IINQBSLFU0AKNY53'
AV_API_KEY_3 = 'L9X61VSB6U2BMPHW'
AV_API_KEY_4 = 'FSMTFWDL6XFDQSLN'
AV_API_KEY_5 = '7FMXJEH0JBVC8D7M'

# keys = [
#    AV_API_KEY,
#    AV_API_KEY_2,
#    AV_API_KEY_3,
#    AV_API_KEY_4,
#    AV_API_KEY_5
# ]

def get_bulk_earnings_data():
   """
   Reads from earnings/ to get diff
   Make sure FINAL_US_STOCKS.csv exists inside stocks/

   """
   remaining =  list(set(read_symbols('stocks/FINAL_US_STOCKS.csv')) - set(parse_symbols('earnings/')))
   print("Remaining", len(remaining))
   with open('logs/earnings_errors.txt', 'w') as f:
    for symbol in remaining:
        try:
          df = av_get_earnings(AV_API_KEY, symbol)
          if df is None:
                raise KeyError()
          df.to_csv(fr'earnings/{symbol}_EARNINGS_DATA.csv', mode='w', sep=',', header=df.columns)
          print("[+] DONE {}".format(symbol))
          time.sleep(math.ceil(12))
        except KeyError as ke:
          print('KeyError: quarterly Earnings missing for ' + str(symbol))
          f.write('KeyError: quarterly Earnings missing for ' + str(symbol))
          pass
        except Exception as e:
          print("An exception occured: {}".format(e))
          f.write(str(symbol) + '\t' + str(e) + '\n')
          pass


def av_get_adjusted_ohlc(api_key, symbol):
    """ return adjusted OHLC for symbol for the year as a dataframe"""
    url = f"{BASE_URL}/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&datatype=json&outputsize=full&apikey={api_key}"

    response = requests.get(url)

    return pd.DataFrame(response.json()['Time Series (Daily)'])

def get_bulk_us_ohlc_data(filename, start_date='', end_date=''):
    symbols = read_symbols(filename=filename)

    for symbol in symbols:
      df = av_get_adjusted_ohlc(api_key=AV_API_KEY, symbol=symbol)
      df = df.T      
      df['date'] = df.index
      if start_date and end_date:
        filtered_df = df.loc[(df['date'] > start_date) & (df['date'] < end_date)]
      else:
        filtered_df = df
      # write to a CSV file
      filtered_df.to_csv(rf"ohlc/{symbol}_ADJUSTED_OHLC.txt", header=filtered_df.columns, sep=',', mode='w')
      time.sleep(12) # limit is 5 requests / minute
      print(f"[+] {symbol} DONE")

def get_us_ohlc_data(symbol, start_date='', end_date=''):
    df = av_get_adjusted_ohlc(api_key=AV_API_KEY, symbol=symbol)
    df = df.T      
    df['date'] = df.index
    if start_date and end_date:
      filtered_df = df.loc[(df['date'] > start_date) & (df['date'] < end_date)]
    else:
      filtered_df = df
    # write to a CSV file
    filtered_df.to_csv(rf"tmp/{symbol}_ADJUSTED_OHLC.txt", header=filtered_df.columns, sep=',', mode='w')
    print(f"[+] {symbol} DONE")

def av_get_earnings(api_key, ticker):
    ticker = ticker.strip().upper()

    url = f"{BASE_URL}/query?function=EARNINGS&symbol={ticker}&apikey={api_key}"
    response = requests.get(url)
    raw_data = response.json()
    if 'quarterlyEarnings' in raw_data:
        return pd.DataFrame(raw_data['quarterlyEarnings'])
    else:
        return None

def av_get_income_statement(api_key, symbol):
    url = rf"{BASE_URL}/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)
    frame = pd.DataFrame(response.json()['quarterlyReports'])
    frame.to_csv(rf'income_stmts/{symbol}_INCOME_STATEMENT.csv', mode='w', sep=',', header=frame.columns)


remaining =  list( set(read_symbols('stocks/FINAL_US_STOCKS.csv')) - set(parse_symbols('earnings/')))
print("Remaining", len(remaining))
# get_bulk_earnings_data()
# df = av_get_earnings(AV_API_KEY, "META")
# print(df)
# earnings_data = av_get_earnings(AV_API_KEY, symbol)
# earnings_data.to_csv(f"earnings/{symbol}_EARNINGS_DATA.txt", sep=',', mode='w', header=earnings_data.columns)
# get_bulk_us_ohlc_data("tmp/US_STOCKS.txt")
# get_us_ohlc_data(symbol=symbol)
# get adjusted OHLC data for all US_stocks for the year 2022


Remaining 4324


In [13]:
BASE_URL = r"https://www.alphavantage.co"
AV_API_KEY = 'IHLNUQ0G66C0MJID'  # basil api key

In [14]:
def av_api(func, api_key, symbol):
    assert symbol is not None
    symbol = symbol.strip().upper()

    url = f"{BASE_URL}/query?function={func}&symbol={symbol}&apikey={api_key}"
    
    response = requests.get(url)
    if func == 'EARNINGS':
        return pd.DataFrame(response.json()['quarterlyEarnings'])
    elif func == 'BALANCE_SHEET':
        return pd.DataFrame(response.json()['quarterlyReports'])
    elif func == 'INCOME_STATEMENT':
        return pd.DataFrame(response.json()['quarterlyReports'])

In [16]:
import os

def parse_symbols(dirname):
   files = os.listdir(dirname)
   return [e[:e.index('_')] for e in files]

def read_symbols(filename):
    with open(filename, "r") as file:
      data = file.read().split('\n')
      data = [d.split(',')[1] for d in data]
      data.pop(0) # ignore header
    return data

def get_av_data(symbol):
    #For reported Earnings & Analyst Estimated EPS, totalRevenue, commonStockSharesOutstanding
    df1 = av_api('EARNINGS',AV_API_KEY, symbol)
    df2 = av_api('INCOME_STATEMENT',AV_API_KEY, symbol)[['fiscalDateEnding','totalRevenue']]
    df1 = df1.merge(df2, on='fiscalDateEnding', how='left')
    df3 = av_api('BALANCE_SHEET',AV_API_KEY, symbol)[['fiscalDateEnding','commonStockSharesOutstanding']]
    df1 = df1.merge(df3, on='fiscalDateEnding', how='left')

    return df1

def get_bulk_data():
    symbols = read_symbols('stocks/FINAL_US_STOCKS.csv')
    remaining =  list(set(symbols) - set(parse_symbols('data/')))
    print("remaining", len(remaining))
    with open('logs/earnings_errors.txt', 'w') as f:
        for symbol in remaining:
            try:
                df = get_av_data(symbol)
                df.to_csv(fr'data/{symbol}_DATA.csv', mode='w', sep=',', header=df.columns)
                print("[+] DONE {} of {}".format(remaining.index(symbol), len(remaining)))
                time.sleep(1)
            except KeyError as ke:
                print('KeyError: quarterly Earnings missing for ' + str(symbol))
                f.write('KeyError: quarterly Earnings missing for ' + str(symbol))
                pass
            except Exception as e:
                print("An exception occured: {}".format(e))
                f.write(str(symbol) + '\t' + str(e) + '\n')
                pass


def network_check():
    while True:
        def check_ping():
            hostname = "8.8.8.8"
            response = os.system("ping -c 1 " + hostname)
            if response == 0:
                pingstatus = 0
            else: pingstatus = 1

            return pingstatus
        
        def get_bulk_data():
            symbols = read_symbols('stocks/FINAL_US_STOCKS.csv')
            remaining =  list(set(symbols) - set(parse_symbols('data/')))
            print("remaining", len(remaining))
            with open('logs/earnings_errors.txt', 'w') as f:
                for symbol in remaining:
                    try:
                        df = get_av_data(symbol)
                        df.to_csv(fr'data/{symbol}_DATA.csv', mode='w', sep=',', header=df.columns)
                        print("[+] DONE {} of {}".format(remaining.index(symbol), len(remaining)))
                        time.sleep(1)
                    except KeyError as ke:
                        print('KeyError: quarterly Earnings missing for ' + str(symbol))
                        f.write('KeyError: quarterly Earnings missing for ' + str(symbol))
                        pass
                    except Exception as e:
                        print("An exception occured: {}".format(e))
                        f.write(str(symbol) + '\t' + str(e) + '\n')
                        pass
        
        net_stat = check_ping()

        if net_stat == 0:
            get_bulk_data()
        else:
            print("Resetting network adapters")
            dwnnw = "ifconfig wlan0 down"
            upnw = "ifconfig wlan0 up"
            os.system(dwnnw)
            os.system(upnw)
            time.sleep(1)
            continue

get_bulk_data()

In [ ]:
df

1. Get all OHLC data for 7k companies
2. Get earnings data for each company in ohlc/
3. Get company overview for each company in ohlc/
4. 

In [49]:
import datetime

# def get_bulk_us_earnings_data(filename):
#     """
#     filename should be 'stocks/FINAL_US_STOCKS.csv
#     """
#     symbols = read_symbols(filename=filename)

#     for symbol in symbols:
#         earnings_data = av_get_earnings(api_key=AV_API_KEY, ticker=symbol)
#         earnings_data.to_csv(
#             f"earnings/{symbol}_EARNINGS_DATA.txt", sep=',', mode='w', header=earnings_data.columns)
#         time.sleep(12)

def av_get_company_overview(api_key, symbol):
    """
    deprecated
    """
    symbol = symbol.strip().upper()

    url = f"{BASE_URL}/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)

    return response.json()


def get_bulk_us_overview(filename):
    symbols = read_symbols(filename=filename)

    for symbol in symbols:
        overview = av_get_company_overview(api_key=AV_API_KEY, symbol=symbol)
        with open(f"overview/{symbol}_OVERVIEW.json", "w") as f:
            f.write(overview)
        time.sleep(12)


def build_gap_table(filename):
    """
    deprecated
    """
    symbols = read_symbols(filename=filename)

    for symbol in symbols:
        ohlc_df = pd.read_csv(f"ohlc/{symbol}_ADJUSTED_OHLC.txt")
        earnings_dates = pd.read_csv(f"earnings/{symbol}_EARNINGS_DATA.txt")
        ohlc_df = ohlc_df.set_index('date')
        openings_at_ex_date_df = ohlc_df.loc[ohlc_df.index.isin(
            earnings_dates)]
        overview_json = pd.read_json(f"overview/{symbol}_OVERVIEW.txt")

        myrows = []
        # for each earnings date
        for index, row in openings_at_ex_date_df.iterrows():
            # find the closing date data before exdate
            if ohlc_df.index[ohlc_df['Unnamed: 0'] == row['Unnamed: 0']].tolist()[0]:
                c1 = ohlc_df.iloc[ohlc_df.index.get_loc(
                    ohlc_df.index[ohlc_df['Unnamed: 0'] == row['Unnamed: 0']].tolist()[0])+1]
            mydict = {}
            mydict['exdate'] = index
            mydict['Opening at ex date'] = row['1. open']
            mydict['C1 date'] = c1['Unnamed: 0']
            mydict['C1'] = c1['4. close']
            mydict['Gap%'] = (row['1. open'] / c1['4. close'] - 1) * 100
            mydict['Market Cap'] = overview_json['MarketCapitalization']
            mydict['52W_dist'] = c1['4. close'] - float(overview_json['52WeekHigh'])

            fiftytwowh_date_series = ohlc_df.loc[ohlc_df['2. high'] == float(
                overview_json['52WeekHigh'])]['Unnamed: 0']

            # TODO: Refactor
            if fiftytwowh_date_series is not None:
                c1_date = datetime.datetime.strptime(
                    c1['Unnamed: 0'], "%Y-%m-%d").date()
                fiftytwoWH_date = datetime.datetime.strptime(
                    fiftytwowh_date_series[0], "%Y-%m-%d").date()

                # doesn't make sense to calculate recency if NOT from the same year
                if c1_date.year == fiftytwoWH_date.year:
                    recency = c1_date - fiftytwoWH_date
                else:  
                    recency = ''
            else:
                recency = ''
            
            mydict['52W_recency'] = recency
            mydict['Trading vol at exdate'] = row['6. volume']

            myrows.append(mydict)
        gap = pd.DataFrame(myrows)
        gap.to_csv(f"{symbol}_GAP_TABLE.txt", header=gap.columns, sep=',', mode='w')
        print(f"[+] Constructed gap table for ${symbol}")


In [6]:
symbol = "META"
earnings_dates = av_get_earnings(AV_API_KEY, symbol)['reportedDate']
type(earnings_dates)

pandas.core.series.Series

In [8]:
get_us_ohlc_data("META")

[+] META DONE


In [9]:
ohlc_df = pd.read_csv(f"ohlc/{symbol}_ADJUSTED_OHLC.txt")

In [10]:
ohlc_df = ohlc_df.set_index('date')
openings_at_ex_date_df = ohlc_df.loc[ohlc_df.index.isin(earnings_dates)]

In [11]:
def av_get_company_overview(api_key, symbol):
    symbol = symbol.strip().upper()
    
    url = f"{BASE_URL}/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)

    return response.json()

In [12]:
overview_json = av_get_company_overview(AV_API_KEY, "META")

In [43]:
symbol = "META"
def get_52_week_data(date):
    """
    Return the 52 week (252 days) 
    Note: This function ignores the first 51 weeks of the historical OHLC as 52Wh is not defined
    """
    myrows = []
    for i in range(252):
      try:
        c1 = ohlc_df.iloc[ohlc_df.index.get_loc(ohlc_df.index[ohlc_df['Unnamed: 0'] == date].tolist()[0]) + i]
        myrows.append(c1)
      except IndexError:
         pass
    frame = pd.DataFrame(myrows)
    high = max(frame['2. high'])
    high_date = frame['2. high'].idxmax()
    low_date = frame['3. low'].idxmin()
    low = min(frame['3. low'])
    wh52 = frame.index.get_indexer_for([high_date])
    wl52 = frame.index.get_indexer_for([low_date])
    return (high, int(wh52), low, int(wl52))

def get_65_day_low(date):
    myrows = []
    ohlc_df = pd.read_csv(fr"ohlc/{symbol}_ADJUSTED_OHLC.txt")
    for i in range(65):
      try:
        c1 = ohlc_df.iloc[ohlc_df.index.get_loc(ohlc_df.index[ohlc_df['Unnamed: 0'] == date].tolist()[0]) + i]
        myrows.append(c1)
      except IndexError:
        pass
    return min(pd.DataFrame(myrows)['4. close'])
get_65_day_low('2023-02-01')

88.91

1. Ex date
2. O
3. H
4. L
5. C
6. C1 date
7. C1
8. Gap%
9. Mkt. Cap
10. 52WH dist
11. 52WH recency
12. 52WL dist
13. 52WL recency
14. 65DL dist
15. Day of the week
16. Avg 20 days volume
17. v0
18. v1
19. fundamentals

In [32]:
def get_avg_volume(symbol="META", start="", days=20):
    if start is None:
      return None
    ohlc_df = pd.read_csv(fr"ohlc/{symbol}_ADJUSTED_OHLC.csv")
    myrows = []
    for i in range(days):
      try:
        vols = ohlc_df.iloc[ohlc_df.index.get_loc(ohlc_df.index[ohlc_df['Unnamed: 0'] == start].tolist()[0]) + i]
        myrows.append(vols['6. volume'])
      except IndexError:
         pass      
    return pd.Series(myrows).mean()

get_avg_volume("META", "2023-02-01")

28445164.9

In [50]:
earnings_data = pd.read_csv(fr"earnings/{symbol}_EARNINGS_DATA.csv")

In [77]:
def av_get_income_statement(api_key, symbol):
    url = rf"{BASE_URL}/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)
    frame = pd.DataFrame(response.json()['quarterlyReports'])
    frame.to_csv(rf'income_stmts/{symbol}_INCOME_STATEMENT.csv', mode='w', sep=',', header=frame.columns)

def get_fundamental_data(symbol, exdate):
    earnings = pd.read_csv(fr"earnings/{symbol}_EARNINGS_DATA.txt")
    income_stmts = pd.read_csv(fr"income_stmts/{symbol}_INCOME_STATEMENT.csv")
    row = earnings[earnings['reportedDate'] == exdate]

    reported_eps = row[['reportedEPS', 'estimatedEPS', 'surprise']]
    # return type(float(reported_eps))
    return reported_eps
    # return (reported_eps, estimated_eps, surprise)
    # return (float(reported_eps), float(estimated_eps), float(surprise))
get_fundamental_data('META','2023-02-01')


,reportedEPS,estimatedEPS,surprise
0,1.76,2.22,-0.46


In [83]:
import datetime
import calendar
myrows = []
for index, row in openings_at_ex_date_df.iterrows():
    if ohlc_df.index[ohlc_df['Unnamed: 0'] == row['Unnamed: 0']].tolist()[0]:
        c1 = ohlc_df.iloc[ohlc_df.index.get_loc(
            ohlc_df.index[ohlc_df['Unnamed: 0'] == row['Unnamed: 0']].tolist()[0])+1]
    mydict = {}
    mydict['exdate'] = index
    mydict['weekday'] = calendar.day_name[datetime.datetime.strptime(index, "%Y-%m-%d").date().weekday()]
    mydict['O'] = row['1. open']
    mydict['H'] = row['2. high']
    mydict['L'] = row['3. low']
    mydict['C'] = row['4. close']
    mydict['C1 date'] = c1['Unnamed: 0']
    mydict['C1'] = c1['4. close']
    mydict['Gap%'] = (row['1. open'] / c1['4. close'] - 1) * 100

    # TODO: Market cap
    # 1. Market cap = Total shares outstanding * price at that date
    mydict['MktCap'] = float(overview_json['SharesOutstanding']) * float(row['1. open'])

    # TODO: Compute 52WH from OHLC data (52*7)
    # Formula: (52WH - Close) / 52WH
    WH_52, WH52_recency, WL_52, WL52_recency = get_52_week_data(c1['Unnamed: 0'])
    DL_65 = get_65_day_low(c1['Unnamed: 0'])
    # print(WH_52, WH52_recency, WL_52, WL52_recency, "C1", c1['Unnamed: 0'])

    mydict['52WH_dist'] = (WH_52 - c1['4. close']) / WH_52
    mydict['52WH_recency'] = WH52_recency
    mydict['52WL_dist'] = (c1['4. close'] - WH_52) / WL_52
    mydict['52WL_recency'] = WL52_recency

    # TODO: check validity
    mydict['65DL_dist'] = (c1['4. close'] - DL_65) / DL_65

    # TODO: 52WH_recency
    # Redo with respect to new defintion
    # fiftytwowh_date_series = ohlc_df.loc[ohlc_df['2. high'] == float(
    #     overview_json['52WeekHigh'])]['Unnamed: 0']
    # if fiftytwowh_date_series is not None:
    #     c1_date = datetime.datetime.strptime(
    #         c1['Unnamed: 0'], "%Y-%m-%d").date()
    #     fiftytwoWH_date = datetime.datetime.strptime(
    #         fiftytwowh_date_series[0], "%Y-%m-%d").date()
    #     if c1_date.year == fiftytwoWH_date.year:
    #         recency = c1_date - fiftytwoWH_date
    #     else:
    #         recency = ''
    # else:
    #     recency = ''


    mydict['v0'] = row['6. volume']
    mydict['v1'] = c1['6. volume']
    mydict['avg_v20'] = get_avg_volume(symbol="META", start=c1['Unnamed: 0'], days=20)
    # TODO: Get vol1 as well

    # FUNDAMENTALS
    fundamental_data = get_fundamental_data(symbol='META', exdate=row['Unnamed: 0'])
    # print(type(fundamental_data['estimatedEPS']))
    mydict['reportedEPS'] = fundamental_data['reportedEPS']
    mydict['estimatedEPS'] = fundamental_data['estimatedEPS']
    mydict['surprise'] = fundamental_data['surprise']

    myrows.append(mydict)

gap = pd.DataFrame(myrows)


In [84]:
gap

,exdate,weekday,O,H,L,C,C1 date,C1,Gap%,MktCap,...,52WH_recency,52WL_dist,52WL_recency,65DL_dist,v0,v1,avg_v20,reportedEPS,estimatedEPS,surprise
0,2023-02-01,Wednesday,148.030,153.5800,147.0600,153.120,2023-01-31,148.9700,-0.631000,3.294793e+11,...,249,-2.032353,58,0.675515,55661017,29842109,2.743854e+07,"0 1.76 Name: reportedEPS, dtype: float64","0 2.22 Name: estimatedEPS, dtype: object","0 -0.46 Name: surprise, dtype: object"
1,2022-10-26,Wednesday,131.680,135.5500,128.5300,129.820,2022-10-25,137.5100,-4.239692,2.930881e+11,...,232,-1.765444,8,0.084806,89294679,38433219,3.400374e+07,"1 1.64 Name: reportedEPS, dtype: float64","1 1.89 Name: estimatedEPS, dtype: object","1 -0.25 Name: surprise, dtype: object"
2,2022-07-27,Wednesday,162.590,170.8700,162.0200,169.580,2022-07-26,159.1500,2.161483,3.618863e+11,...,225,-1.459838,22,0.021174,43846557,28468424,2.595283e+07,"2 2.46 Name: reportedEPS, dtype: float64","2 2.59 Name: estimatedEPS, dtype: object","2 -0.13 Name: surprise, dtype: object"
3,2022-04-27,Wednesday,174.430,181.2099,169.0000,174.950,2022-04-26,180.9500,-3.603205,3.882393e+11,...,163,-1.133036,0,0.000000,71062886,30872853,2.655493e+07,"3 2.72 Name: reportedEPS, dtype: float64","3 2.56 Name: estimatedEPS, dtype: object","3 0.16 Name: surprise, dtype: object"
4,2022-02-02,Wednesday,327.820,328.0000,316.8747,323.000,2022-02-01,319.0000,2.764890,7.296486e+11,...,105,-0.257712,231,0.082714,58458280,18023796,2.151016e+07,"4 3.67 Name: reportedEPS, dtype: float64","4 3.84 Name: estimatedEPS, dtype: object","4 -0.17 Name: surprise, dtype: object"
5,2021-10-25,Monday,320.300,329.5600,319.7200,328.690,2021-10-22,324.6100,-1.327747,7.129109e+11,...,36,-0.244144,195,0.002594,38408957,35224543,2.171985e+07,"5 3.22 Name: reportedEPS, dtype: float64","5 3.19 Name: estimatedEPS, dtype: object","5 0.03 Name: surprise, dtype: object"
6,2021-07-28,Wednesday,374.560,377.5499,366.9300,373.280,2021-07-27,367.8100,1.835187,8.336807e+11,...,2,-0.032838,249,0.215700,29676910,15705447,1.464411e+07,"6 3.61 Name: reportedEPS, dtype: float64","6 3.03 Name: estimatedEPS, dtype: object","6 0.58 Name: surprise, dtype: object"
7,2021-04-28,Wednesday,307.360,310.9200,305.3700,307.100,2021-04-27,303.5700,1.248476,6.841096e+11,...,13,-0.067430,251,0.191920,33907210,15309256,1.707433e+07,"7 3.3 Name: reportedEPS, dtype: float64","7 2.37 Name: estimatedEPS, dtype: object","7 0.93 Name: surprise, dtype: object"
8,2021-01-27,Wednesday,282.530,283.4500,268.1201,272.140,2021-01-26,282.0500,0.170183,6.288440e+11,...,104,-0.164989,216,0.148225,35346194,19373636,2.063690e+07,"8 3.88 Name: reportedEPS, dtype: float64","8 3.22 Name: estimatedEPS, dtype: object","8 0.66 Name: surprise, dtype: object"
9,2020-10-29,Thursday,276.550,283.6000,273.7800,280.830,2020-10-28,267.6700,3.317518,6.155339e+11,...,44,-0.269876,156,0.147370,32368118,23121808,1.872545e+07,"9 2.71 Name: reportedEPS, dtype: float64","9 1.91 Name: estimatedEPS, dtype: object","9 0.8 Name: surprise, dtype: object"


In [16]:
gap.to_csv("tmp/META_GAP_TABLE.csv", mode='w', sep=',', header=gap.columns)

In [18]:
gap = pd.read_csv(rf"tmp/{symbol}_GAP_TABLE.csv")

In [19]:
gap

,Unnamed: 0,exdate,O,C1 date,C1,Gap%,MktCap,52WH_dist,52WL_dist,52WH_recency,52WL_recency,v0,v1
0,0,2023-02-01,148.030,2023-01-31,148.9700,-0.631000,3.294793e+11,0.612390,-2.671813,355,58,55661017,29842109
1,1,2022-10-26,131.680,2022-10-25,137.5100,-4.239692,2.930881e+11,0.642209,-2.014362,289,8,89294679,38433219
2,2,2022-07-27,162.590,2022-07-26,159.1500,2.161483,3.618863e+11,0.585903,-1.459838,225,22,43846557,28468424
3,3,2022-04-27,174.430,2022-04-26,180.9500,-3.603205,3.882393e+11,0.529181,-1.133036,163,0,71062886,30872853
4,4,2022-02-02,327.820,2022-02-01,319.0000,2.764890,7.296486e+11,0.169984,-0.267603,105,344,58458280,18023796
5,5,2021-10-25,320.300,2021-10-22,324.6100,-1.327747,7.129109e+11,0.155387,-0.292573,36,363,38408957,35224543
6,6,2021-07-28,374.560,2021-07-27,367.8100,1.835187,8.336807e+11,0.020036,-0.054850,2,342,29676910,15705447
7,7,2021-04-28,307.360,2021-04-27,303.5700,1.248476,6.841096e+11,0.038970,-0.089788,13,279,33907210,15309256
8,8,2021-01-27,282.530,2021-01-26,282.0500,0.170183,6.288440e+11,0.074244,-0.164989,104,216,35346194,19373636
9,9,2020-10-29,276.550,2020-10-28,267.6700,3.317518,6.155339e+11,0.121443,-0.269876,44,156,32368118,23121808
